In [1]:
import numpy as np
import pandas as pd
import string
import re #regular expression
import random
import pickle

In [2]:
with open("businessCard.txt", mode="r", encoding="utf8", errors="ignore") as f:
    text = f.read()

In [3]:
print(text[:480])

id	text	tag
000.jpeg	 	O
000.jpeg	.	O
000.jpeg	040-4852	B-PHONE
000.jpeg	"8881,"	I-PHONE
000.jpeg	90309	B-PHONE
000.jpeg	52549	I-PHONE
000.jpeg	Fi	O
000.jpeg	/laurelsoverseaseducation	O
000.jpeg	@:	O
000.jpeg	LAURELS	B-ORG
000.jpeg	OVERSEAS	I-ORG
000.jpeg	EDUCATIONAL	I-ORG
000.jpeg	CONSULTANCY	I-ORG
000.jpeg	PVT.	I-ORG
000.jpeg	LTD.	I-ORG
000.jpeg	Sea	O
000.jpeg	|	O
000.jpeg	U.K	O
000.jpeg	AUSTRALIA	O
000.jpeg	CANADA	O
000.jpeg	IRELAND	O
000.jpeg	 	O
000.jpeg	 	O
000.jpeg	 	O


In [4]:
text[:450]

'id\ttext\ttag\n000.jpeg\t \tO\n000.jpeg\t.\tO\n000.jpeg\t040-4852\tB-PHONE\n000.jpeg\t"8881,"\tI-PHONE\n000.jpeg\t90309\tB-PHONE\n000.jpeg\t52549\tI-PHONE\n000.jpeg\tFi\tO\n000.jpeg\t/laurelsoverseaseducation\tO\n000.jpeg\t@:\tO\n000.jpeg\tLAURELS\tB-ORG\n000.jpeg\tOVERSEAS\tI-ORG\n000.jpeg\tEDUCATIONAL\tI-ORG\n000.jpeg\tCONSULTANCY\tI-ORG\n000.jpeg\tPVT.\tI-ORG\n000.jpeg\tLTD.\tI-ORG\n000.jpeg\tSea\tO\n000.jpeg\t|\tO\n000.jpeg\tU.K\tO\n000.jpeg\tAUSTRALIA\tO\n000.jpeg\tCANADA\tO\n000.jpeg\tIRELAND\tO\n000.jpeg'

In [5]:
data = list(map(lambda x: x.split("\t"), text.split("\n")))

In [6]:
data[:20]

[['id', 'text', 'tag'],
 ['000.jpeg', ' ', 'O'],
 ['000.jpeg', '.', 'O'],
 ['000.jpeg', '040-4852', 'B-PHONE'],
 ['000.jpeg', '"8881,"', 'I-PHONE'],
 ['000.jpeg', '90309', 'B-PHONE'],
 ['000.jpeg', '52549', 'I-PHONE'],
 ['000.jpeg', 'Fi', 'O'],
 ['000.jpeg', '/laurelsoverseaseducation', 'O'],
 ['000.jpeg', '@:', 'O'],
 ['000.jpeg', 'LAURELS', 'B-ORG'],
 ['000.jpeg', 'OVERSEAS', 'I-ORG'],
 ['000.jpeg', 'EDUCATIONAL', 'I-ORG'],
 ['000.jpeg', 'CONSULTANCY', 'I-ORG'],
 ['000.jpeg', 'PVT.', 'I-ORG'],
 ['000.jpeg', 'LTD.', 'I-ORG'],
 ['000.jpeg', 'Sea', 'O'],
 ['000.jpeg', '|', 'O'],
 ['000.jpeg', 'U.K', 'O'],
 ['000.jpeg', 'AUSTRALIA', 'O']]

In [7]:
df = pd.DataFrame(data[1:], columns=data[0])

df.head()

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE


In [8]:
string.whitespace

' \t\n\r\x0b\x0c'

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
whitespace = string.whitespace
punctuation = '!#$%&\'()*+:;<=>?[\\]^`{|}~'

tableWhiteSpace = str.maketrans("", "", whitespace)
tablePunctuation = str.maketrans("", "", punctuation)

def clean_text(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhiteSpace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [11]:
df["text"] = df["text"].apply(clean_text)

In [12]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

dataClean.head(10)

C:\Users\my\AppData\Local\Temp\ipykernel_5824\644399253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@,O
9,000.jpeg,laurels,B-ORG
10,000.jpeg,overseas,I-ORG


## Converting data to SpaCy Format

In [13]:
group = dataClean.groupby(by="id")

In [14]:
group.groups.keys()

dict_keys(['000.jpeg', '001.jpeg', '002.jpeg', '003.jpeg', '004.jpeg', '007.jpeg', '008.jpeg', '009.jpeg', '010.jpeg', '011.jpeg', '012.jpeg', '013.jpeg', '014.jpeg', '015.jpeg', '016.jpeg', '017.jpeg', '018.jpeg', '020.jpeg', '021.jpeg', '022.jpeg', '023.jpeg', '024.jpeg', '025.jpeg', '027.jpeg', '028.jpeg', '030.jpeg', '031.jpeg', '032.jpeg', '033.jpeg', '034.jpeg', '035.jpeg', '036.jpeg', '037.jpeg', '038.jpeg', '039.jpeg', '040.jpeg', '041.jpeg', '042.jpeg', '043.jpeg', '044.jpeg', '045.jpeg', '047.jpeg', '048.jpeg', '049.jpeg', '050.jpeg', '051.jpeg', '052.jpeg', '053.jpeg', '054.jpeg', '055.jpeg', '056.jpeg', '057.jpeg', '058.jpeg', '059.jpeg', '060.jpeg', '061.jpeg', '062.jpeg', '063.jpeg', '064.jpeg', '065.jpeg', '066.jpeg', '067.jpeg', '068.jpeg', '069.jpeg', '070.jpeg', '071.jpeg', '072.jpeg', '073.jpeg', '074.jpeg', '075.jpeg', '076.jpeg', '078.jpeg', '079.jpeg', '080.jpeg', '081.jpeg', '082.jpeg', '083.jpeg', '084.jpeg', '085.jpeg', '086.jpeg', '087.jpeg', '088.jpeg', '089.

In [15]:
group.get_group("000.jpeg")

,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@,O
9,000.jpeg,laurels,B-ORG
10,000.jpeg,overseas,I-ORG


In [16]:
grouparray = group.get_group("000.jpeg")[["text","tag"]].values
grouparray

array([['.', 'O'],
       ['040-4852', 'B-PHONE'],
       ['"8881,"', 'I-PHONE'],
       ['90309', 'B-PHONE'],
       ['52549', 'I-PHONE'],
       ['fi', 'O'],
       ['/laurelsoverseaseducation', 'O'],
       ['@', 'O'],
       ['laurels', 'B-ORG'],
       ['overseas', 'I-ORG'],
       ['educational', 'I-ORG'],
       ['consultancy', 'I-ORG'],
       ['pvt.', 'I-ORG'],
       ['ltd.', 'I-ORG'],
       ['sea', 'O'],
       ['u.k', 'O'],
       ['australia', 'O'],
       ['canada', 'O'],
       ['ireland', 'O'],
       ['www.laurelseducation.com', 'B-WEB'],
       ['info@laurelseducation.com', 'B-EMAIL']], dtype=object)

In [17]:
content = ""
annotations = {"entities": []}
start = 0
end = 0

for text, label in grouparray:
    #print(text, label)
    text = str(text)
    stringLength = len(text) + 1
    
    start = end
    end = start + stringLength
    
    if label != "O":
        annot = (start, end-1, label)
        annotations["entities"].append(annot)
        
    content = content +  text + " "

In [18]:
content

'. 040-4852 "8881," 90309 52549 fi /laurelsoverseaseducation @ laurels overseas educational consultancy pvt. ltd. sea u.k australia canada ireland www.laurelseducation.com info@laurelseducation.com '

In [19]:
annotations

{'entities': [(2, 10, 'B-PHONE'),
  (11, 18, 'I-PHONE'),
  (19, 24, 'B-PHONE'),
  (25, 30, 'I-PHONE'),
  (62, 69, 'B-ORG'),
  (70, 78, 'I-ORG'),
  (79, 90, 'I-ORG'),
  (91, 102, 'I-ORG'),
  (103, 107, 'I-ORG'),
  (108, 112, 'I-ORG'),
  (146, 170, 'B-WEB'),
  (171, 196, 'B-EMAIL')]}

In [20]:
print(f'start: {content.find("040-4852")} end: {content.find("040-4852") + len("040-4852")}')

start: 2 end: 10


### Applying to all Dataset

In [21]:
cards = group.groups.keys()
cards

dict_keys(['000.jpeg', '001.jpeg', '002.jpeg', '003.jpeg', '004.jpeg', '007.jpeg', '008.jpeg', '009.jpeg', '010.jpeg', '011.jpeg', '012.jpeg', '013.jpeg', '014.jpeg', '015.jpeg', '016.jpeg', '017.jpeg', '018.jpeg', '020.jpeg', '021.jpeg', '022.jpeg', '023.jpeg', '024.jpeg', '025.jpeg', '027.jpeg', '028.jpeg', '030.jpeg', '031.jpeg', '032.jpeg', '033.jpeg', '034.jpeg', '035.jpeg', '036.jpeg', '037.jpeg', '038.jpeg', '039.jpeg', '040.jpeg', '041.jpeg', '042.jpeg', '043.jpeg', '044.jpeg', '045.jpeg', '047.jpeg', '048.jpeg', '049.jpeg', '050.jpeg', '051.jpeg', '052.jpeg', '053.jpeg', '054.jpeg', '055.jpeg', '056.jpeg', '057.jpeg', '058.jpeg', '059.jpeg', '060.jpeg', '061.jpeg', '062.jpeg', '063.jpeg', '064.jpeg', '065.jpeg', '066.jpeg', '067.jpeg', '068.jpeg', '069.jpeg', '070.jpeg', '071.jpeg', '072.jpeg', '073.jpeg', '074.jpeg', '075.jpeg', '076.jpeg', '078.jpeg', '079.jpeg', '080.jpeg', '081.jpeg', '082.jpeg', '083.jpeg', '084.jpeg', '085.jpeg', '086.jpeg', '087.jpeg', '088.jpeg', '089.

In [22]:
allCardsData = []

for card in cards:
    
    cardData = []
    grouparray = group.get_group(card)[["text","tag"]].values
    content = ""
    annotations = {"entities": []}
    start = 0
    end = 0

    for text, label in grouparray:
        #print(text, label)
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != "O":
            annot = (start, end-1, label)
            annotations["entities"].append(annot)

        content = content +  text + " "
    
    cardData = (content, annotations)
    allCardsData.append(cardData)

In [23]:
allCardsData[:3]

[('. 040-4852 "8881," 90309 52549 fi /laurelsoverseaseducation @ laurels overseas educational consultancy pvt. ltd. sea u.k australia canada ireland www.laurelseducation.com info@laurelseducation.com ',
  {'entities': [(2, 10, 'B-PHONE'),
    (11, 18, 'I-PHONE'),
    (19, 24, 'B-PHONE'),
    (25, 30, 'I-PHONE'),
    (62, 69, 'B-ORG'),
    (70, 78, 'I-ORG'),
    (79, 90, 'I-ORG'),
    (91, 102, 'I-ORG'),
    (103, 107, 'I-ORG'),
    (108, 112, 'I-ORG'),
    (146, 170, 'B-WEB'),
    (171, 196, 'B-EMAIL')]}),
 ('john smith marketing manager web www.psdgraphics.com phone 123-456-7890 mail email@psdgraphics.com ',
  {'entities': [(0, 4, 'B-NAME'),
    (5, 10, 'I-NAME'),
    (11, 20, 'B-DES'),
    (21, 28, 'I-DES'),
    (33, 52, 'B-WEB'),
    (59, 71, 'B-PHONE'),
    (77, 98, 'B-EMAIL')]}),
 ('sau 0 98489 24441 dy "08672," 224441 /enkateswapa wie ',
  {'entities': [(6, 11, 'B-PHONE'), (12, 17, 'I-PHONE'), (37, 49, 'B-ORG')]})]

### Splitting Data

In [24]:
random.shuffle(allCardsData)

allCardsData[:2]

[('ee highveldcoal jonathan erskine managing director bass executive "suites," 1st floor virginia field "house," virginia "airport," durban "north," 4051 south africa "p.0," box "20672," durban "north," 4016 m.27 72539 6214 t. 27 31563 6611 f. 27 31 563 5433 / 27 86 559 2577 jono@highveldcoal.com ',
  {'entities': [(3, 15, 'B-ORG'),
    (16, 24, 'B-NAME'),
    (25, 32, 'I-NAME'),
    (33, 41, 'B-DES'),
    (42, 50, 'I-DE'),
    (205, 209, 'B-PHONE'),
    (210, 215, 'I-PHONE'),
    (216, 220, 'I-PHONE'),
    (224, 226, 'B-PHONE'),
    (227, 232, 'I-PHONE'),
    (233, 237, 'I-PHONE'),
    (273, 294, 'B-EMAIL')]}),
 ('friend unlock 34 696 900 91 pc www.trustfriendunlock.com trustfriendunlock@hotmail.com admin@trustfriendunlock.com ',
  {'entities': [(14, 16, 'B-PHONE'),
    (17, 20, 'I-PHONE'),
    (21, 24, 'I-PHONE'),
    (25, 27, 'I-PHONE'),
    (31, 56, 'B-WEB'),
    (57, 86, 'B-EMAIL'),
    (87, 114, 'I-EMAIL')]})]

In [25]:
len(allCardsData)

267

In [26]:
trainData = allCardsData[:240]
testData = allCardsData[240:]

### Save Data

In [27]:
pickle.dump(trainData, open("data/trainData.pickle", mode="wb"))
pickle.dump(testData, open("data/testData.pickle", mode="wb"))